In [1]:
print("Harsh")

Harsh


In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from ctransformers import AutoModelForCausalLM
import time
import os
import pypdf

e:\DevStuff\Medical Chatbot\End-to-End-Medical-Chatbot-using-llama-2\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    
    return documents

In [3]:
extracted_data = load_pdf("data/")

Creating text chunks

In [5]:
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks


In [6]:
text_chunk = text_splitter(extracted_data)
print("Length of the chunk:" , len(text_chunk)) 

Length of the chunk: 5859


Downloaind Embedding model

In [7]:
def downloading_embeddings_model():
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings_model

In [9]:
embeddings = downloading_embeddings_model()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello World")
print("Dimension of Vector created by embedding models:",len(query_result))

Dimension of Vector created by embedding models: 384


e:\DevStuff\Medical Chatbot\End-to-End-Medical-Chatbot-using-llama-2\mchatbot\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [12]:
from pinecone import Pinecone
from langchain_pinecone import Pinecone as pine_cone
pinecone_apikey = "pcsk_2w4sQk_Rbbxy2NeyJhong1KWnQ4HJrpyLiuTKwqAahqYcAqkXG5smfs9eij44VeCoexwi2"
pc = Pinecone(pinecone_apikey)
index = pc.Index("medical-chatbot")

In [13]:
import os

# Set the Pinecone API key as an environment variable before using langchain_pinecone
os.environ["PINECONE_API_KEY"] = pinecone_apikey

docsearch = pine_cone.from_texts(
    [t.page_content for t in text_chunk],
    embeddings,
    index_name="medical-chatbot"
)

e:\DevStuff\Medical Chatbot\End-to-End-Medical-Chatbot-using-llama-2\mchatbot\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [14]:
docsearch = pine_cone.from_existing_index("medical-chatbot",embeddings)

query = "What are Allergies"

docs =docsearch.similarity_search(query,k=3)
print("Result",docs)

Result [Document(id='01f992a0-e611-4422-a212-cb0c10cb3e98', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='97432ff0-a5df-49f2-aadb-a6e4dca14afe', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reaction

In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [28]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [17]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [33]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}  # Make sure this line is included
)

In [21]:
print(qa)

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['"prompt"', 'context', 'question'], input_types={}, partial_variables={}, template='\nUse the following pieces of information to answer the user\'s question.\nIf you don\'t know PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])\nchain_type_kwargs={"prompt": PROMPT}the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n'), llm=CTransformers(client=<ctransformers.llm.LLM object at 0x00000283847752D0>, model='model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8}), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={},

In [ ]:
while True:
    user_input = input("Input Prompt: ")
    result = qa({"query": user_input})  # Use "query" instead of "question"
    print("Response : ", result["result"])

e:\DevStuff\Medical Chatbot\End-to-End-Medical-Chatbot-using-llama-2\mchatbot\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Response :  There are many different types of acne, including:
* Comedonal acne: This type of acne occurs when dead skin cells and other debris clog pores, causing blackheads and whiteheads.
* Inflammatory acne: This type of acne occurs when bacteria enter the pore and cause inflammation, leading to redness, swelling, and painful bumps.
Note: The above answer is based on the information provided in the reference text.
